# TM10007 Assignment template

In [53]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [54]:
# Data loading functions. Uncomment the one you want to use
# Import other classifiers you plan to use
from worclipo.load_data import load_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from scipy.stats import ttest_ind
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA


data = load_data()
print(f"The number of samples: {len(data.index)}")
print(f"The number of columns: {len(data.columns)}")
print(f"The number of liposarcoma in the dataset: {len(data[data['label'] == 'liposarcoma'])}")
print(f"The number of lipoma in the dataset: {len(data[data['label'] == 'lipoma'])}")

The number of samples: 115
The number of columns: 494
The number of liposarcoma in the dataset: 58
The number of lipoma in the dataset: 57


Now we know the number of samples, colums and labels. The next step is to look for missing data. 


In [55]:
def split_data(data):

    # Separate features and target variable
    x = data.drop(['label'], axis=1)
    y = data['label']
    
    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test


# Call the function
x_train, x_test, y_train, y_test = split_data(data)

# Optionally, print the sizes of the splits to verify
print("Training set size:", x_train.shape)
print("Testing set size:", x_test.shape)

Training set size: (92, 493)
Testing set size: (23, 493)


In [56]:
def find_zeros_in_data(data):
    # Maak een kopie van de data om de originele data niet te wijzigen
    data_copy = data.copy()
    
    # Vervang waarden die gelijk zijn aan 0 met NaN om ze gemakkelijk te tellen
    data_copy = data_copy.replace(0, np.nan)
    
    # Tel het aantal NaNs in elke kolom, wat overeenkomt met het originele aantal 0's
    zeros_count = data_copy.isna().sum()
    
    # Print het aantal 0's gevonden in elke kolom
    print("Aantal 0's per feature:\n", zeros_count)
    
    # Return het aantal 0's per feature voor verdere analyse indien nodig
    return zeros_count

# Roep de functie aan met je dataset 'data'
zeros_count = find_zeros_in_data(data)


Aantal 0's per feature:
 label                                                       0
PREDICT_original_sf_compactness_avg_2.5D                    0
PREDICT_original_sf_compactness_std_2.5D                    0
PREDICT_original_sf_rad_dist_avg_2.5D                       0
PREDICT_original_sf_rad_dist_std_2.5D                       0
                                                         ... 
PREDICT_original_phasef_phasesym_peak_position_WL3_N5     115
PREDICT_original_phasef_phasesym_range_WL3_N5               0
PREDICT_original_phasef_phasesym_energy_WL3_N5              0
PREDICT_original_phasef_phasesym_quartile_range_WL3_N5     12
PREDICT_original_phasef_phasesym_entropy_WL3_N5             0
Length: 494, dtype: int64


Feature normalization

In [57]:
def scaling_data(x_train,x_test):
    scaler = preprocessing.StandardScaler().fit(x_train)

    # Applying scaler to train and test set
    X_train_scaled = pd.DataFrame(scaler.transform(x_train), index=x_train.index, columns=x_train.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(x_test), index=x_test.index, columns=x_test.columns)

    return X_train_scaled, X_test_scaled

X_train_scaled, X_test_scaled = scaling_data(x_train, x_test)


Feature selection 

In [58]:
def manual_feature_selection_ttest(X_train_scaled, y_train):
    '''defiition to calculate the Ttest and P-value of every feature
    input: x_train, y_train
    output: beste 5 festures'''
    
    lipoma = X_train_scaled[y_train =='lipoma']
    liposarcoma = X_train_scaled[y_train =='liposarcoma']

    Ttest_indResult = pd.DataFrame(columns=['t_statistic', 'p_value'])
    for feature in X_train_scaled.columns:
        t_statistic, p_value = ttest_ind(lipoma[feature], liposarcoma[feature])
        feature_result = pd.DataFrame({'t_statistic': t_statistic, 'p_value': p_value}, index = [feature])
        Ttest_indResult = pd.concat([Ttest_indResult, feature_result])

    absolute_features = Ttest_indResult['t_statistic'].abs()
    best_5_features = absolute_features.nlargest(5)
    best_5_features_index = best_5_features.index.tolist()
    df_best_5_features = pd.DataFrame(X_train_scaled[best_5_features_index])
    return df_best_5_features

df_best_5_features = manual_feature_selection_ttest(X_train_scaled, y_train)

C:\Users\31642\AppData\Local\Temp\ipykernel_7812\174714973.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Ttest_indResult = pd.concat([Ttest_indResult, feature_result])
c:\Users\31642\miniconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\31642\miniconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*sa

In [59]:
desired_variance = 0.50 # 80% gewenste verklaarde variantie
pca = PCA(n_components=desired_variance)

pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

Classifiers 

- decision trees
- naive bayes
- linear classifiers
- quadratic classifiers
- ⁠nearest neighbour
- random forrest 
- ⁠support vector machines
- neural network

In [49]:
clf_rfc = RandomForestClassifier()
param_grid = {
    'n_estimators': range(10, 311, 100),  # Number of trees in the forest
    'max_depth': range(10,51,10),      # Maximum depth of the trees
    'min_samples_split': range(2,11,4),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': range(1,14,4)}  # Minimum number of samples required to be at a leaf node

grid_search = GridSearchCV(estimator=clf_rfc, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(df_best_5_features, y_train)

# Get the best hyperparameters
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best accuracy score: {grid_search.best_score_}")


Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 310}
Best accuracy score: 0.7853801169590644


In [50]:
clf_svm = SVC()

param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear', 'poly'],
              'degree': [1, 3, 5]}

grid_search = GridSearchCV(estimator=clf_svm, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(df_best_5_features, y_train)

# Get the best hyperparameters
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best accuracy score: {grid_search.best_score_}")


Best Hyperparameters: {'C': 100, 'degree': 1, 'gamma': 1, 'kernel': 'linear'}
Best accuracy score: 0.7087719298245614


In [51]:
clf_knn = KNeighborsClassifier()

param_grid = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1, 2]}  # 1 for Manhattan distance, 2 for Euclidean distance

grid_search = GridSearchCV(estimator=clf_knn, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(df_best_5_features, y_train)

# Get the best hyperparameters
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best accuracy score: {grid_search.best_score_}")



Best Hyperparameters: {'algorithm': 'auto', 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Best accuracy score: 0.7076023391812866


c:\Users\31642\miniconda3\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [0.6005848  0.62280702 0.57777778 0.58947368 0.67660819 0.67719298
 0.6748538  0.67426901 0.68596491 0.70760234 0.6754386  0.66491228
 0.64152047 0.67426901 0.70643275 0.70701754 0.6005848  0.62280702
 0.57777778 0.58947368 0.67660819 0.67719298 0.6748538  0.67426901
 0.68596491 0.70760234 0.6754386  0.66491228 0.64152047 0.67426901
 0.70643275 0.70701754 0.6005848  0.62280702 0.57777778 0.58947368
 0.67660819 0.67719298 0.6748538  0.67426901 0.68596491 0.70760234
 0.6754386  0.66491228 0.64152047 0.67426901 0.70643275 0.70701754
        nan 0.62280702        nan 0.58947368        nan 0.67719298
        nan 0.67426901        nan 0.70760234        nan 0.66491228
        nan 0.67426901        nan 0.70701754]
  warnings.warn(


In [52]:
clf_dt = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [None, 10, 20, 30],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],
              'max_features': ['auto', 'sqrt', 'log2']}

grid_search = GridSearchCV(estimator=clf_dt, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(df_best_5_features, y_train)

# Get the best hyperparameters
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best accuracy score: {grid_search.best_score_}")


Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}
Best accuracy score: 0.7619883040935672


c:\Users\31642\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
262 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\31642\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\31642\miniconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\31642\miniconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\31642\miniconda3\Lib\site-

In [60]:
clf_mlp = MLPClassifier()

# Create the hidden layer size variable
size_hidden_layers = []
for a in range(5,100,10):
    for b in range(5,100,10):
        size_hidden_layers.append((a,b))

param_grid = {'hidden_layer_sizes': size_hidden_layers,
              'activation': ['identity', 'logistic', 'tanh', 'relu'],
              'solver': ['lbfgs', 'sgd','adam'],
              'alpha': [0.000001, 0.00001, 0.0001, 0.001, 0.01],
              'learning_rate': ['constant', 'invscaling','adaptive'],
              'max_iter': range(100,1001,100)}

mlp_search = GridSearchCV(estimator=clf_mlp, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=5)
mlp_search.fit(df_best_5_features,y_train)

# Get the best hyperparameters
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best accuracy score: {grid_search.best_score_}")


KeyboardInterrupt: 